# Compile and summarize news with MultiOn

In this recipe, we'll demonstrate how to find the latest news and summarize and analyze it using MultiOn Agents. We'll be using `step` and `retrieve`.

## Step 1: Set up the environment

First, let's install the required libraries and set up the MultiOn client.

In [ ]:
%pip install multion --upgrade

In [1]:
from multion.client import MultiOn

client = MultiOn(
    api_key="YOUR_API_KEY" # Get your API key from https://app.multion.ai/api-keys
)

## Step 2: Create the news article search agent

Next, we will create a remote agent session (without local mode), which will be responsible for searching for the top news articles.

In [2]:
create_response = client.sessions.create(
    url="https://www.google.com",
    include_screenshot=True
)

session_id = create_response.session_id

Now, we can continue from the same session using the `session_id` from the response. We will search for news articles and use `step` until we get to the results page.

In [ ]:
status = "CONTINUE"

while status == "CONTINUE":
    step_response = client.sessions.step(
        session_id=session_id,
        cmd="Search for the top 5 news articles on google. Don't open a news website.",
        include_screenshot=True
    )
    status = step_response.status

print(step_response.message)
print(step_response.screenshot)

## Step 3: Retrieve article headlines and URLs.

Next, we will call `retrieve` on the same session in order to get the news article headlines, publisher names and URLs.

In [ ]:
retrieve_response = client.retrieve(
    session_id=session_id,
    cmd="Get the news articles with their headlines, publishers and urls.",
    fields=["headline", "publisher", "url"],
    scroll_to_bottom=True,
    render_js=True
)

data = retrieve_response.data
top_5 = data[:5]

## Step 4: Summarize and analyze articles in parallel

Next, we will call `retrieve` in parallel on each article (this creates new sessions instead of continuing with the same one) to extract a summary and analyze the impact on our fictional business at the same time.

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import pprint

def summarize_article(article):
    summary = ""
    try:
        article_summary_response = client.retrieve(
            url=article["url"],
            cmd="I run a chip manufacturing business. Get the headline and a summary for the article. Also analyze whether this news might have an impact on my business.",
            fields=["headline", "summary", "impact"]
        )
        summary = article_summary_response.data[0]
    except:
        pass
    return summary

with ThreadPoolExecutor() as executor:
    summaries = list(executor.map(summarize_article, top_5))

print(f"Article summaries and impact:")
pprint.pp(summaries)